# 이어드림 4기 모의대회 1차 특강

In [11]:
# import os

# os.chdir('/content/drive/MyDrive/2024_이어드림/모의1차')

In [12]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import pandas as pd

train = pd.read_csv('../data/train.csv')
X = train.drop('depvar',axis=1)
y = train['depvar']

X_train, X_valid, y_train, y_valid = train_test_split(X, y , test_size=0.2)

In [13]:
print(len(train), len(X_train), len(X_valid))

100000 80000 20000


In [14]:
skf = StratifiedKFold(random_state=42, shuffle=True)
for i, (train_index, valid_index) in enumerate(skf.split(X,y)):
  print(f"Fold {i}:")
  print(f"   Train : length={len(train_index)} / indices]{train_index[:10]}")
  print(f"   Valid : length={len(valid_index)} / indices]{valid_index[:10]}")

Fold 0:
   Train : length=80000 / indices][ 1  3  4  7  8  9 10 11 12 14]
   Valid : length=20000 / indices][ 0  2  5  6 13 15 18 38 43 46]
Fold 1:
   Train : length=80000 / indices][ 0  2  3  4  5  6  7 10 11 12]
   Valid : length=20000 / indices][ 1  8  9 17 19 26 27 32 40 42]
Fold 2:
   Train : length=80000 / indices][ 0  1  2  5  6  7  8  9 11 12]
   Valid : length=20000 / indices][ 3  4 10 14 20 28 31 33 51 52]
Fold 3:
   Train : length=80000 / indices][0 1 2 3 4 5 6 7 8 9]
   Valid : length=20000 / indices][16 24 25 29 30 34 35 36 41 49]
Fold 4:
   Train : length=80000 / indices][ 0  1  2  3  4  5  6  8  9 10]
   Valid : length=20000 / indices][ 7 11 12 21 22 23 37 39 44 57]


In [15]:
train.head()

,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,pub_rec,revol_bal,total_acc,collections_12_mths_ex_med,acc_now_delinq,...,term1,open_acc,installment,revol_util,out_prncp,out_prncp_inv,total_rec_int,fico_range_low,fico_range_high,depvar
0,0.0824,21000.0,29.19,0,1,0,3016,26,0,0,...,1,18,37.74,0.076,0.0,0.0,157.94,765,769,0
1,0.1299,80000.0,4.82,0,1,1,5722,24,0,0,...,1,8,269.52,0.447,0.0,0.0,1702.42,665,669,0
2,0.1299,38000.0,23.66,0,3,0,6511,18,0,0,...,1,7,168.45,0.880,0.0,0.0,1066.64,670,674,0
3,0.1367,100000.0,16.27,4,2,0,6849,30,0,0,...,1,12,510.27,0.457,0.0,0.0,1256.24,680,684,1
4,0.1269,30000.0,25.28,0,1,2,8197,12,0,0,...,1,8,335.45,0.416,0.0,0.0,871.04,660,664,1


# Model Selection
lazypredict
...

# parameter tuning

In [16]:
# !pip install optuna

In [17]:

import optuna

In [18]:
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

def objective(trial, data = train.iloc[:,:-1], target = train.iloc[:,-1]):
  train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.2, random_state=42)
  param = {
        'tree_method':'hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

  model = XGBClassifier(**param)
  model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], early_stopping_rounds=10, verbose=False)
  preds = model.predict(valid_x)

  return f1_score(valid_y, preds, average='macro')

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2024-06-03 21:31:34,249] A new study created in memory with name: no-name-880ffc98-ee27-4136-9a18-b2dbfec3a690
/var/folders/3h/jprthrkx73n0d9pl0mgr0kqm0000gn/T/ipykernel_33623/1163693986.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
/var/folders/3h/jprthrkx73n0d9pl0mgr0kqm0000gn/T/ipykernel_33623/1163693986.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
/opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibili

# train-val KFold

In [20]:
from xgboost import XGBClassifier

f1 = []
for i, (train_index, valid_index) in enumerate(skf.split(X,y)):

  train_X, train_y = train.iloc[train_index,:-1], train.iloc[train_index,-1]
  valid_X, valid_y = train.iloc[valid_index,:-1], train.iloc[valid_index,-1]

  xgb_clf = XGBClassifier(**study.best_trial.params)
  xgb_clf.fit(train_X, train_y,eval_set = [(valid_X, valid_y)], early_stopping_rounds=10, verbose=False)

  pred_valid = xgb_clf.predict(valid_X)
  f1.append(f1_score(valid_y, pred_valid, average='macro'))

f1.mean() # fold f1_macro mean

/opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages/xgboost/sklearn.py:

AttributeError: 'list' object has no attribute 'mean'

In [ ]:
test = pd.read_csv('../data/test.csv')
ss = pd.read_csv('../data/sample_submission.csv')
final_model = XGBClassifier(**study.best_trial.params)
result = final_model.predict(test)
ss['answer'] = result
# ss.to_csv('xgb_optuna_f5.csv', index=False)